In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from random import seed
from random import randrange

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#the dataset aims to predict the whether a given banknote is authentic given a number of measures taken from a photograph. 
#it is a binary classification problem where 0 is the label for an authentic banknote while 1 for a fake one.
banknoteData = pd.read_csv('/content/drive/MyDrive/MachineLearning/datasets/data_banknote_authentication.csv')
banknoteData.head()

,Variance of Wavelet Transformed image,Skewness of Wavelet Transformed image,Kurtosis of Wavelet Transformed image,Entropy of image,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
X = banknoteData.iloc[:,:-1].values
y = (banknoteData.iloc[:,-1].values).reshape(-1,1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
print('n samples in Train set: ', len(X_train))
print('n samples in Test set: ', len(X_test))

n samples in Train set:  1097
n samples in Test set:  275


#Random Forest from scratch#

*random forest relies on the bagging methods which is an ensembling technique. The idea is to divide the training dataset in subsets and then train several decision trees separately, one for each subset.Once all predictors are trained, the ensemble can make a prediction for a new instance by aggregating the predictions of all predictors.
The term bagging stands for bootstrap aggregation.
The bagging method is characterized by the fact that the sampling is performed with replacement, i.e.the same row (set of input vatriables with the corresponding input) may be chosen and added to the sample more than once.*


reference: https://carbonati.github.io/posts/random-forests-from-scratch/#:~:text=Random%20forests%20are%20essentially%20a,us%20with%20a%20powerful%20classifier.

In [ ]:
#entropy loss: it measures the uncertainty (inpurity). It is minimized when all samples in a node belong to the same class
#it is maximized when we have uniform class distribution

#p is the probability P(X=1) that a banknote is a true one
def entropy (p):
  if p==0 or p==1: entropy = 0
  else:
    #- (p * np.log2(p) + (1 - p) * np.log2(1-p))
    entropy = -(p * np.log2(p) + (1 - p) * np.log2(1-p))
    #-p*np.log2(p)-(1-p)*np.log2(1-p)
  return entropy

#after a slit we compare the entropy before the latter and after it. The measure we get is the information gain. 
#infomation gain reveals how much information when splitting a node at a particular value
#the idea is to split a node with the value that maximizes the information gain(IG)

#IG function takes of the classes from the left and right child and returns the information gain of that particular split.
def IG(left_child,right_child):

  parent = left_child+right_child
  N_p = len(parent)
  N_left = len(left_child)
  N_right = len(right_child)

  if len(parent) > 0:
    #.count(1) counts the number of elements with class=1
    #in this phase p for parent,left and right child is computed
    p_parent = parent.count(1)/N_p
  else: p_parent = 0

  if len(left_child) > 0:
    p_lf_c = left_child.count(1)/N_left
  else: p_lf_c = 0

  if len(right_child) > 0:
    p_rt_c = right_child.count(1)/N_right
  else: p_rt_c = 0

  #compute entropy for parent and left and right child
  ID_p = entropy(p_parent)
  ID_left = entropy(p_lf_c)
  ID_right = entropy(p_rt_c)
  #information gain 
  IG = ID_p - (N_left/N_p)*ID_left -(N_right/N_p)*ID_right
  return IG 

In [ ]:
#each decision tree is trained on a bootstrapped subset coming from our dataset. 
# considering that the dataset has n samples, bootstrapping is the process of sampling n point with replacement (i.e., some obsverations in our dataset will be selected more than once and some won't be selected at all)

def bootstrap(X,y):
  #takes random indices from the original train frame
  bootstrap_idxs = list(np.random.choice(range(X.shape[0]),X.shape[0], replace = True))
  oob_idxs = []
  for i in range(X.shape[0]):
    if i not in bootstrap_idxs:
       oob_idxs.append(i)
  #select the samples of the corresponding indices
  X_bootstrap = X[bootstrap_idxs]
  y_bootstrap = y[bootstrap_idxs]
  
  X_oob = X[oob_idxs]
  y_oob = y[oob_idxs]

  return X_bootstrap, y_bootstrap, X_oob, y_oob

#X_bootstrap, y_bootstrap, X_oob, y_oob = bootstrap(X_train,y_train)

#OOB = out-of-bag error estimate: the remaing samples that were not selected to build a parcticular tree.
#once we've built our tree with the n bootstrapped observations we can test each x_i that was left out and compute the mean prediction error from those samples.
#for each tree we can compute an OOB score  and take the average of all those scores to get an estimate for how accurate our model performs

def OOB(tree, X, y):
  mis_label = 0
  for i in range(X.shape[0]):
    pred = single_tree_pred(tree, X[i])
    if pred != y[i]:
      mis_label += 1
  return mis_label / len(X)

In [ ]:
def get_split_point(X_b,y_b,max_features):
  fts_list = list()
  n_features = len(X_b[0])
  
  #select n features at random 
  while len(fts_list) <= max_features:
    feature_idx = random.sample(range(n_features),1)
    if feature_idx not in fts_list:
        fts_list.extend(feature_idx)
  
  #for each feature selected, iterate through each value in the bootstrapped dataset and compute the information gain
  best_IG=-999
  node = None
  for idx in fts_list:
    for split_point in X_b[:,idx]:
      left_child = {'X_bootstrap': [], 'y_bootstrap': []}
      right_child = {'X_bootstrap': [], 'y_bootstrap': []}

      # split children for continuous variables
      if type(split_point) in [int, float]:
        for i, val in enumerate(X_b[:,idx]):
          if val <= split_point:
            left_child['X_bootstrap'].append(X_b[i])
            left_child['y_bootstrap'].append(y_b[i])
          else:
            right_child['X_bootstrap'].append(X_b[i])
            right_child['y_bootstrap'].append(y_b[i])
      # split children for categoric variables
      else:
        for j, value in enumerate(X_b[:,idx]):
          if value == split_point:
            left_child['X_bootstrap'].append(X_b[j])
            left_child['y_bootstrap'].append(y_b[j])
          else:
            right_child['X_bootstrap'].append(X_b[j])
            right_child['y_bootstrap'].append(y_b[j])

      split_IG = IG(left_child['y_bootstrap'], right_child['y_bootstrap'])
      #Return a dictionary from the value that gives the highest information gain
      if split_IG>best_IG:
        best_IG = split_IG
        left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
        right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
        node = {'information_gain': split_IG,
                'left_child': left_child,
                'right_child': right_child,
                'split_point': split_point,
                'feature_idx': idx}


  return node

In [ ]:
def terminal_node(node):
  y_b = node['y_bootstrap']
  pred = max(y_b, key=y_b.count)
  return pred

def split_node(node, max_features, min_samples_split, max_depth, depth):
  #remove left and right child from the original dictionary
  left_child = node['left_child']
  right_child = node['right_child']
  del(node['left_child'])
  del(node['right_child'])

  #check if the children has no observations. 
  #If one of the children is entirely empty this ultimately means that the best split in the data for that node was unable to differentiate the 2 classes 
  #therefore, its best to call terminal_node and return the tree. terminal_node returns the class with the highest counts at the current node.
  if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
    empty_child = {'y_bootstrap': left_child['y_bootstrap']+right_child['y_bootstrap']}
    node['left_split'] = terminal_node(empty_child)
    node['right_split'] = terminal_node(empty_child)
    return

  #if the current depth = max_depth then create a terminal node and return the tree
  if depth >= max_depth:
    node['left_split'] = terminal_node(left_child)
    node['right_split'] = terminal_node(right_child)
    return node

  #check if min_split is greater than the number of observation in the left child at the current node in order to make a split
  if len(left_child['X_bootstrap']) <= min_samples_split:
    node['left_split'] = node['right_split'] = terminal_node(left_child)
  else:
    node['left_split'] = get_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
    split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth+1)
  
  if len(right_child['X_bootstrap']) <= min_samples_split:
    node['right_split'] = node['left_split'] = terminal_node(right_child)
  else:
    node['right_split'] = get_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
    split_node(node['right_split'], max_features, min_samples_split, max_depth, depth+1)


In [ ]:
#definition of a single tree
def single_tree(X_b, y_b, max_depth, min_samples_split, max_features):
  root_node = get_split_point(X_b, y_b, max_features)
  split_node(root_node, max_features, min_samples_split, max_depth, 1)
  return root_node

#random forest paramters
#n_tree = number of tree in the model
#max_features= n of features to consider when looking for the best split
#max_depth = the max depth of a tree
#min_sample_split=  the min n. of samples required to split an internal node
def random_forest(X,y, n_tree, max_features, max_depth, min_samples_split):
  tree_ls = list()
  oob_ls = list()
  for i in range(n_tree):
    X_bootstrap, y_bootstrap, X_oob, y_oob = bootstrap(X, y)
    tree = single_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
    tree_ls.append(tree)
    oob_error = OOB(tree, X_oob, y_oob)
    oob_ls.append(oob_error)
  print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
  return tree_ls

In [ ]:
#given a input vector we can predict its class given a single tree 
def single_tree_pred(tree, X):
  ft_idx = tree['feature_idx']
  if X[ft_idx] <= tree['split_point']:
    if type(tree['left_split']) == dict:
      return single_tree_pred(tree['left_split'], X)
    else:
      value = tree['left_split']
      return value
  else:
    if type(tree['right_split']) == dict:
      return single_tree_pred(tree['right_split'], X)
    else:
      return tree['right_split']

def predict_rf(tree_ls, X):
  pred_ls = list()
  for i in range(len(X)):
    ensemble_preds = [single_tree_pred(tree, X[i]) for tree in tree_ls]
    final_pred = max(ensemble_preds, key = ensemble_preds.count)
    pred_ls.append(final_pred)
  return np.array(pred_ls)

**Train**

In [ ]:
n_tree = 100
max_features = 3
max_depth = 10
min_samples_split = 2

rf = random_forest(X_train, y_train, n_tree, max_features, max_depth, min_samples_split)

OOB estimate: 0.46


**Test**

In [ ]:
preds = predict_rf(rf, X_test)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

Testing accuracy: [0.607]


*Comparison with the Slearn Random Forest*

In [6]:
from sklearn.ensemble import RandomForestClassifier

sk_rand_for = RandomForestClassifier(n_estimators=100, max_leaf_nodes=2, n_jobs=-1)
#train
y_t = y_train.ravel()
sk_rand_for.fit(X_train, y_t)

RandomForestClassifier(max_leaf_nodes=2, n_jobs=-1)

In [7]:
#test phase
y_pred_sk = sk_rand_for.predict(X_test)
cm = confusion_matrix(y_test, y_pred_sk)
print('Random Forest confusion matrix: \n', cm)
print ("Accuracy : ", accuracy_score(y_test, y_pred_sk))
print(classification_report(y_test, y_pred_sk))

Random Forest confusion matrix: 
 [[138  15]
 [ 27  95]]
Accuracy :  0.8472727272727273
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       153
           1       0.86      0.78      0.82       122

    accuracy                           0.85       275
   macro avg       0.85      0.84      0.84       275
weighted avg       0.85      0.85      0.85       275

